# TP2 - Préparation des Données

 Ce notebook charge les deux jeux de données (Hi-Seq et ECG), effectue la normalisation nécessaire et sauvegarde les fichiers préparés (`.npy` ou `.csv`) pour qu'ils soient utilisés par les autres notebooks de clustering et de détection d'anomalies.

In [12]:
# --- Imports de base ---
from pathlib import Path
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings

In [18]:
PROJECT_ROOT = Path("..")

# --- Chemins des données ---
DATA_PATH = PROJECT_ROOT / "data"

DATA_FILE1 = "hiseq_data.csv"
DATA_FILE_PATH1 = DATA_PATH / DATA_FILE1

DATA_FILE2 = "hiseq_labels.csv"
DATA_FILE_PATH2 = DATA_PATH / DATA_FILE2

# --- Chemin des fonctions utilitaires ---
PROJECT_FCT = Path("../src").resolve()
sys.path.append(str(PROJECT_FCT))

# Chemin des données traitées (en sortie)
DATA_PROCESSED_PATH = PROJECT_ROOT / "data"

print(f"Chemin du projet racine : {PROJECT_ROOT.resolve()}")
print(f"Tentative de chargement du fichier : {DATA_PATH.resolve()}")
print(f"Sauvegarde vers : {DATA_PROCESSED_PATH}")

Chemin du projet racine : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git
Tentative de chargement du fichier : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git\data
Sauvegarde vers : ..\data


In [14]:
try:
    from utils import load_dataset
    from preprocess import normalize_data
except ImportError:
    print("Attention: Les fichiers tp2_utils.py ou tp2_preprocess.py n'ont pas été trouvés.")
    print("Définition de fonctions de remplacement.")

warnings.filterwarnings('ignore')

# 1. Préparation des données Hi-Seq (Clustering)

 - Chargement des données et des étiquettes (vérité terrain).
 - Normalisation (StandardScaler).
 - Sauvegarde de `X_scaled` et `y` pour les notebooks de clustering.

In [15]:
# Charger les données 
X_hiseq, y_hiseq = load_dataset(DATA_FILE_PATH1, DATA_FILE_PATH2)

print(f"Forme de X_hiseq: {X_hiseq.shape}")
print(f"Forme de y_hiseq: {y_hiseq.shape}")

Chargement des données Hi-Seq data
Données Hi-Seq chargées, forme de X: (801, 20531)
Chargement des données Hi-Seq labels
Données Hi-Seq labels chargées, forme de y: (801, 1)
Forme de X_hiseq: (801, 20531)
Forme de y_hiseq: (801,)


In [16]:
# --- Normalisation Hi-Seq ---
X_hiseq_scaled = normalize_data(X_hiseq)

Normalisation des données avec StandardScaler...


In [19]:
# --- Sauvegarde Hi-Seq ---
print(f"Sauvegarde des fichiers Hi-Seq normalisés dans {DATA_PROCESSED_PATH}...")

# Utilise le chemin de sortie pour la sauvegarde
np.save(DATA_PROCESSED_PATH / 'hiseq_X_scaled.npy', X_hiseq_scaled)
y_hiseq.to_csv(DATA_PROCESSED_PATH / 'hiseq_y_labels.csv', header=True)

print("Terminé.")

Sauvegarde des fichiers Hi-Seq normalisés dans ..\data...
Terminé.


# 2. Préparation des données ECG (Détection d'Anomalies)

 - Chargement des données ECG.
 - Séparation des données normales (0) et anormales (1).
 - Création des ensembles:
    - `Train` : 60% des données normales (pour l'entraînement des modèles).
    - `Validation` : 10% des normales + 20% des anormales (pour trouver le seuil).
    - `Test` : 30% des normales + 80% des anormales (pour l'évaluation finale).
 - Normalisation (StandardScaler) *entraîné seulement sur `X_train`*.
 - Sauvegarde des 6 fichiers (`X_train_scaled`, `y_train`, `X_val_scaled`, `y_val`, `X_test_scaled`, `y_test`).